In [ ]:
pip install stardist

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from stardist import export_imagej_rois, random_label_cmap
from stardist.models import StarDist2D
import imageio
from skimage.measure import label, regionprops
import os
from csbdeep.utils import Path, normalize

# Input folder containing the images
input_folder = 'D:/IMPART/NEW_METHOD_TRIAL/MANY/MANY IMAGES/Input_Images/Input'
output_base_folder = 'D:/IMPART/NEW_METHOD_TRIAL/MANY/MANY IMAGES/Input_Images/Output'

# Ensure that the base output folder exists
os.makedirs(output_base_folder, exist_ok=True)

# Initialize the StarDist model
model = StarDist2D.from_pretrained('2D_versatile_he')

# Iterate over each input image
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        # Read the input image
        img = imageio.imread(os.path.join(input_folder, filename))
        
        # Make predictions using the model
        he_labels, _ = model.predict_instances(normalize(img))
        
        # Save the prediction visualization
        plt.rcParams["figure.figsize"] = (20,10)
        plt.subplot(1,2,1)
        plt.imshow(img, cmap="gray")
        plt.axis("off")
        plt.title("Input image")
        
        plt.subplot(1,2,2)
        plt.imshow(he_labels)
        plt.axis("off")
        plt.title("Prediction")
        
        plt.savefig(os.path.join(output_base_folder, f'{os.path.splitext(filename)[0]}_prediction.png'))
        plt.close()
        
        # Label the segmented nuclei
        labeled_nuclei = label(he_labels)
        
        # Iterate over each labeled nucleus and save it as a separate image
        for region in regionprops(labeled_nuclei):
            y0, x0, y1, x1 = region.bbox
            nucleus = img[y0:y1, x0:x1]
            nucleus_filename = os.path.join(output_base_folder, f'nucleus_{os.path.splitext(filename)[0]}_{region.label}.png')
            imageio.imwrite(nucleus_filename, nucleus)
